In [29]:
import argparse
import cv2
import numpy as np

from yolo import YOLO


print("loading yolo-tiny...")
yolo = YOLO("models/cross-hands-tiny.cfg", "models/cross-hands-tiny.weights", ["hand"])

yolo.size = int(416)
yolo.confidence = float(0.2)

print("starting webcam...")
cv2.namedWindow("preview")
vc = cv2.VideoCapture('http://192.168.0.100:4747/video')

if vc.isOpened():  # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    width, height, inference_time, results = yolo.inference(frame)

    results.sort(key=lambda x: x[2])

    hand_count = len(results)

    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    dilated_img = cv2.dilate(grayFrame, np.ones((3,3), np.uint8))
    bg_img = cv2.medianBlur(dilated_img, 21)

    _, thr_img = cv2.threshold(255 - cv2.absdiff(grayFrame, bg_img), 230, 0, cv2.THRESH_TRUNC)
    
    cv2.normalize(thr_img, thr_img, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    
    for detection in results[:hand_count]:
        id, name, confidence, x, y, w, h = detection
        cx = x + (w / 2)
        cy = y + (h / 2)

        color = (255, 255, 255)
        cv2.rectangle(thr_img, (x, y), (x + w, y + h), (255, 255, 255), -1)
    
    diff_img = cv2.resize(thr_img, (1080,640), interpolation = cv2.INTER_CUBIC)
    
    _,diff_img = cv2.threshold(diff_img,235,255,cv2.THRESH_TOZERO_INV)
    
    cv2.imshow("preview", diff_img)

    rval, frame = vc.read()

    key = cv2.waitKey(20)
    if key == 27:  # exit on ESC
        break

cv2.destroyWindow("preview")
vc.release()


loading yolo-tiny...
starting webcam...
